In [22]:
import numpy as np

import slowquant.SlowQuant as sq
from slowquant.unitary_coupled_cluster.ucc_wavefunction import WaveFunctionUCC

import pyscf
from pyscf import mcscf, ao2mo


geometry = """H 0.0 0.0 0.0; H 0.0 0.0 0.7414;""" #Angstrom
basis = "cc-pvdz"
cas = (2, 4) # active space
unit = "angstrom"  #angstrom, bohr

#### SlowQuant
SQobj = sq.SlowQuant()
SQobj.set_molecule(geometry, distance_unit=unit,)
SQobj.set_basis_set(basis)

#### PySCF
mol = pyscf.M(atom = geometry, basis = basis, unit = unit)
mf = mol.RHF().run()

#### HF 
mo_coeffs = mf.mo_coeff
print(mo_coeffs)
print(mo_coeffs.shape)
print(len(mo_coeffs))

#### get the integrals in the AO basis
hcore_ao = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
eri_4fold_ao = mol.intor('int2e_sph')
print(hcore_ao)

# OO-UCCSD
WF = WaveFunctionUCC(
num_elec=SQobj.molecule.number_electrons,
cas=cas,
mo_coeffs=mo_coeffs,
h_ao=hcore_ao,
g_ao=eri_4fold_ao,
excitations="SD",
include_active_kappa=True,
)

WF.run_wf_optimization_1step(
		optimizer_name="SLSQP", 
		tol = 1e-8,
		orbital_optimization=True)



print("oo-UCCSD energy =",WF.energy_elec)
print()
print("Now let's try to run it again but using the integrals in the MO basis, and not the AO.") 
print()

hcore_mo = np.einsum('pi,pq,qj->ij', mo_coeffs, hcore_ao, mo_coeffs)
eri_4fold_mo = ao2mo.incore.full(eri_4fold_ao, mo_coeffs)
mo_coeffs_ = np.identity(len(mo_coeffs))

# OO-UCCSD
WF_ = WaveFunctionUCC(
num_elec=SQobj.molecule.number_electrons,
cas=cas,
mo_coeffs=mo_coeffs_,
h_ao=hcore_mo,
g_ao=eri_4fold_mo,
excitations="SD",
include_active_kappa=True,
)

WF_.run_wf_optimization_1step(
		optimizer_name="SLSQP", 
		tol = 1e-8,
		orbital_optimization=True)

print("oo-UCCSD energy in MO representation = ",WF_.energy_elec)
print()
print("difference = ", WF.energy_elec-WF_.energy_elec)


converged SCF energy = -1.12871495902965
[[ 4.03501999e-01  1.50502995e-01 -7.00543341e-01  7.62451001e-01
   1.58873793e-16  7.58112947e-17 -4.43487532e-01  1.39917967e-17
   8.23162232e-18  2.67196903e+00]
 [ 1.74091016e-01  1.95445051e+00  7.26419413e-01 -1.61644630e+00
   6.63722945e-16 -1.76816481e-16  2.20947623e-01 -3.77615568e-16
   4.59683190e-17  5.31887858e-02]
 [ 5.49471929e-17 -2.05717554e-16  1.14131507e-16  1.99586012e-17
   5.76204235e-01  5.98174862e-02 -6.16108365e-16 -6.76111021e-01
   7.23259325e-01 -1.17280162e-16]
 [-4.49532740e-18 -7.46966497e-19  4.02813693e-17  2.39026766e-18
  -5.98174862e-02  5.76204235e-01  5.40925306e-17  7.23259325e-01
   6.76111021e-01  2.81631433e-18]
 [ 2.24771816e-02  1.41989040e-02 -1.15208819e-02 -4.09030003e-01
   5.12607437e-16  1.34404559e-17  7.26015064e-01 -4.05321332e-16
   3.27501470e-16  2.02289180e+00]
 [ 4.03501999e-01 -1.50502995e-01 -7.00543341e-01 -7.62451001e-01
  -3.26292858e-16  1.51822927e-16 -4.43487532e-01  3.73589

In [29]:
import os
from qiskit_nwchem_driver.nwchem2yaml import extract_fields
from qiskit_nwchem_driver import nwchem_driver
import yaml



data_dir = os.path.join("..","data","PW_LiH_data", "3x3_periodic", "NWChem",'1covo')
datafile = os.path.join(data_dir, "H1Li1-2.0.out")
data_file_yaml = os.path.join(data_dir, "H1Li1-2.0.yaml")

data = extract_fields(datafile)
with open(data_file_yaml, 'w') as f:
    f.write(yaml.dump(data, default_flow_style=False)) 

driver = nwchem_driver.NWchem_Driver(data_file_yaml)

n_electrons, n_spatial_orbitals, nuclear_repulsion_energy, h1, h2 = driver.load_from_yaml(data_file_yaml, include_spin=True)
#print(h1)
#print(h2)
#print(h2.shape)




In [30]:
mo_coeffs_ = np.identity(4)

WF_ = WaveFunctionUCC(
num_elec=n_electrons,
cas=(2,2),
mo_coeffs=mo_coeffs_,
h_ao=h1,
g_ao=h2,
excitations="SD",
include_active_kappa=True,
)

WF_.run_wf_optimization_1step(
		optimizer_name="SLSQP", 
		tol = 1e-8,
		orbital_optimization=True)

print("oo-UCCSD energy in MO representation = ",WF_.energy_elec)
print()
#print("difference = ", WF.energy_elec-WF_.energy_elec)

### Parameters information:
### Number kappa: 5
### Number theta1: 1
### Number theta2: 1
### Number theta3: 0
### Number theta4: 0
### Number theta5: 0
### Number theta6: 0
### Total parameters: 7

Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------     1      |         0.01       |     -0.8745725707061084    
--------     2      |         0.01       |     -1.1058175009457594    
--------     3      |         0.01       |     -1.1059777948737630    
--------     4      |         0.01       |     -1.1059885631198427    
--------     5      |         0.01       |     -1.1059886098492586    
--------     6      |         0.01       |     -1.1059886098492586    
oo-UCCSD energy in MO representation =  -1.1059886098492586

